In [15]:
import pandas as pd
import numpy as np
import ta

In [16]:
forex_data = pd.read_csv('EURUSD_M30-1.csv')
forex_data.head(5)

,Time,Open,High,Low,Close,Volume
0,2025-04-09 16:30,1.10273,1.10412,1.10132,1.10200,4807
1,2025-04-09 16:00,1.10386,1.10423,1.10245,1.10274,5592
2,2025-04-09 15:30,1.10620,1.10691,1.10369,1.10381,7612
3,2025-04-09 15:00,1.10404,1.10631,1.10365,1.10619,8314
4,2025-04-09 14:30,1.10431,1.10512,1.10258,1.10404,9421


In [17]:
forex_data['Time'] = pd.to_datetime(forex_data['Time'], format='%Y-%m-%d %H:%M')
forex_data.dtypes

Time      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [18]:
forex_data = forex_data.sort_values(by ='Time', ascending=False)
forex_data.columns = forex_data.columns.str.strip().str.capitalize()
forex_data

,Time,Open,High,Low,Close,Volume
0,2025-04-09 16:30:00,1.10273,1.10412,1.10132,1.10200,4807
1,2025-04-09 16:00:00,1.10386,1.10423,1.10245,1.10274,5592
2,2025-04-09 15:30:00,1.10620,1.10691,1.10369,1.10381,7612
3,2025-04-09 15:00:00,1.10404,1.10631,1.10365,1.10619,8314
4,2025-04-09 14:30:00,1.10431,1.10512,1.10258,1.10404,9421
...,...,...,...,...,...,...
99995,2017-03-31 16:00:00,1.06970,1.06976,1.06906,1.06921,3748
99996,2017-03-31 15:30:00,1.06939,1.06989,1.06892,1.06971,5132
99997,2017-03-31 15:00:00,1.06940,1.07018,1.06889,1.06925,8520
99998,2017-03-31 14:30:00,1.06755,1.06966,1.06720,1.06941,7520


In [ ]:

forex_data['rsi'] = ta.momentum.RSIIndicator(close=forex_data['Close']).rsi()
forex_data['ema_20'] = ta.trend.EMAIndicator(forex_data['Close'], window=20).ema_indicator()
forex_data['ema_9'] = ta.trend.EMAIndicator(close=forex_data['Close'], window=9).ema_indicator()
forex_data['atr'] = ta.volatility.AverageTrueRange(forex_data['High'], forex_data['Low'], forex_data['Close']).average_true_range()
forex_data['buy_signal'] = np.random.choice([0, 1], size=len(forex_data), p=[0.9, 0.1]) 
forex_data['sell_signal'] = np.random.choice([0, 1], size=len(forex_data), p=[0.9, 0.1])
forex_data['rsi_ema_buy_signal'] = (forex_data['rsi'] > 50) & (forex_data['close'] > forex_data['ema_20'])
forex_data['rsi_ema_sell_signal'] = (forex_data['rsi'] < 50) & (forex_data['close'] < forex_data['ema_20'])

forex_data.dropna(axis=1)



#one way to consider prediction to buy/sell
# forex_data['signal'] = 0
# forex_data.loc[forex_data['ema_9'] > forex_data['ema_21'], 'signal'] = 1  # Buy
# forex_data.loc[forex_data['ema_9'] < forex_data['ema_21'], 'signal'] = -1 # 


KeyError: 'rsi_14'

In [7]:
for i in range(1, 4):
    forex_data[f'return_lag_{i}'] = forex_data['Close'].pct_change(i)
# forex_data['hour'] = forex_data['Time'].dt.hour
# forex_data['day_of_week'] = forex_data.index.dayofweek

# forex_data.dropna(inplace=True)


In [8]:
forex_data.columns

Index(['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'rsi', 'ema_20',
       'ema_9', 'atr', 'buy_signal', 'sell_signal', 'return_lag_1',
       'return_lag_2', 'return_lag_3'],
      dtype='object')